In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from progressbar import ProgressBar
import time,os

In [2]:
from lasagne.layers import InputLayer,DenseLayer,batch_norm
from lasagne.nonlinearities import rectify,sigmoid,leaky_rectify,elu
from lasagne.updates import adam

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5005)


In [3]:
from Tars.model import GAN
from Tars.distribution import Bernoulli
from Tars.load_data import mnist

In [4]:
load,plot = mnist('../datasets/')
train_x,_,_,_,test_x,_ = load(test=True)

x_dim       = 28*28
z_dim       = 64
y_dim       = 10

activation = rectify
seed = 1234
np.random.seed(seed)

n_epoch = 100
n_batch = 100

optimizer = adam
optimizer_params={"learning_rate":1e-4, "beta1":0.5}

In [5]:
z = InputLayer((None,z_dim))
p_0  = batch_norm(DenseLayer(z,num_units=512,nonlinearity=activation))
p_1  = batch_norm(DenseLayer(p_0,num_units=512,nonlinearity=activation))
p_mean = DenseLayer(p_1,num_units=x_dim,nonlinearity=sigmoid)
p = Bernoulli(p_mean,given=[z]) #p(x|z)

x = InputLayer((None,x_dim))
d_0  = DenseLayer(x,num_units=512,nonlinearity=leaky_rectify)
d_1  = DenseLayer(d_0,num_units=512,nonlinearity=leaky_rectify)
d_mean = DenseLayer(d_1,num_units=1,nonlinearity=sigmoid)
d = Bernoulli(d_mean,given=[x]) #d(t|x)

In [6]:
model = GAN(p,d,n_batch,optimizer,optimizer,optimizer_params,optimizer_params)

In [ ]:
n_sample = 100
sample_z  = np.random.standard_normal((n_batch, z_dim)).astype(np.float32)

def plot_image(t,i):
    sample_x = p.np_sample_mean_given_x(sample_z)
    fig = plt.figure(figsize=(10,10))
    X,cmap = plot(sample_x[:n_sample])

    for j,x in enumerate(X):
            ax = fig.add_subplot(10, 10, j + 1)
            ax.imshow(x,cmap)
            ax.axis('off')

    plt.savefig('../plot/%d/%04d.jpg'%(t,i))
    plt.close()

In [ ]:
t = int(time.time())
os.mkdir('../plot/%d' % t)

model.set_seed(seed)
pbar = ProgressBar(maxval=n_epoch).start()
for i in range(1, n_epoch+1):
    np.random.shuffle(train_x)
    loss_train = model.train([train_x])
    if (i%10 == 0) or (i == 1):
        loss_test = model.gan_test([test_x])
        lw = "epoch = %d, loss (train) = %lf %lf loss (test) = %lf %lf\n" %(i,loss_train[0],loss_train[1],loss_test[0],loss_test[1])
        f = open("../plot/%d/temp.txt" % t, "a")
        f.write(lw)
        f.close()
        print lw
        plot_image(t,i)
        
    pbar.update(i)

  0% (  0 of 100) |                                                                              | Elapsed Time: 0:00:00 ETA:  --:--:--

epoch = 1, loss (train) = 4.664785 0.086644 loss (test) = 4.184406 0.045174



  9% (  9 of 100) |#######                                                                         | Elapsed Time: 0:00:26 ETA: 0:04:32

epoch = 10, loss (train) = 4.758564 0.111711 loss (test) = 3.568944 0.150903



 19% ( 19 of 100) |###############                                                                 | Elapsed Time: 0:00:56 ETA: 0:03:21

epoch = 20, loss (train) = 3.692257 0.289443 loss (test) = 2.736810 0.305367



 25% ( 25 of 100) |####################                                                            | Elapsed Time: 0:01:15 ETA: 0:03:42